In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd

/home/lorenzoquerol/miniconda3/envs/wssv-recognition/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
AUTOTUNE = tf.data.AUTOTUNE
CONFIG = dict(
    epochs=100,
    learning_rate=1e-4,
    batch_size=4,
    img_shape=(224, 224),
    input_shape=(224, 224, 3),
    num_classes=2,
    dropout_rate=0.6,
    es_patience=10,
    seed_value=42,
)
TRAIN_DIR = "dataset-augmented/train"
VAL_DIR = "dataset-augmented/val"
TEST_DIR = "dataset-augmented/test"
CLASS_NAMES = ["healthy", "wssv"]

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    seed=CONFIG["seed_value"],
    image_size=CONFIG["img_shape"],
    batch_size=CONFIG["batch_size"],
    label_mode="categorical",
).prefetch(buffer_size=AUTOTUNE)

valid_dataset = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    seed=CONFIG["seed_value"],
    image_size=CONFIG["img_shape"],
    batch_size=CONFIG["batch_size"],
    label_mode="categorical",
).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    seed=CONFIG["seed_value"],
    image_size=CONFIG["img_shape"],
    batch_size=CONFIG["batch_size"],
    label_mode="categorical",
).prefetch(buffer_size=AUTOTUNE)

Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.


In [4]:
def create_model():
    base_model = tf.keras.applications.EfficientNetV2B0(
        input_shape=CONFIG["input_shape"],
        include_top=False,
        classes=CONFIG["num_classes"],
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=CONFIG["input_shape"])
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(rate=CONFIG["dropout_rate"], seed=CONFIG["seed_value"])(
        x
    )
    outputs = tf.keras.layers.Dense(units=CONFIG["num_classes"], activation="softmax")(
        x
    )

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG["learning_rate"]),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tfa.metrics.F1Score(
                num_classes=CONFIG["num_classes"],
                average="weighted",
                name="f1_score",
                threshold=0.5,
            ),
            tf.keras.metrics.FalseNegatives(name="false_negatives"),
            tf.keras.metrics.TruePositives(name="true_positives"),
            tf.keras.metrics.FalsePositives(name="false_positives"),
            tf.keras.metrics.TrueNegatives(name="true_negatives"),
        ],
    )

    return model


checkpoint_filepath = "checkpoints_efficientnetv2b0"
latest = tf.train.latest_checkpoint(checkpoint_filepath)

model = create_model()
model.load_weights(latest).expect_partial()

train_images = []
train_labels = []

valid_images = []
valid_labels = []

test_images = []
test_labels = []

for X, y in train_dataset.as_numpy_iterator():
    train_images.append(X)
    train_labels.append(y)

train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)
train_labels = np.argmax(train_labels, axis=1)

for X, y in valid_dataset.as_numpy_iterator():
    valid_images.append(X)
    valid_labels.append(y)

valid_images = np.concatenate(valid_images)
valid_labels = np.concatenate(valid_labels)
valid_labels = np.argmax(valid_labels, axis=1)

for X, y in test_dataset.as_numpy_iterator():
    test_images.append(X)
    test_labels.append(y)

test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)
test_labels = np.argmax(test_labels, axis=1)

In [5]:
# # export results as csv
# df = pd.DataFrame(
#     np.array(results).reshape(1, -1),
#     columns=[
#         "loss",
#         "precision",
#         "recall",
#         "f1_score",
#         "false_negatives",
#         "true_positives",
#         "false_positives",
#         "true_negatives",
#     ],
# )
# df.to_csv("results/efficientnet-test.csv")

In [6]:
train_preds = model.predict(train_images)[:, 1].tolist()
valid_preds = model.predict(valid_images)[:, 1].tolist()
test_preds = model.predict(test_images)[:, 1].tolist()

train_preds_df = pd.DataFrame(
    columns=["actual", "predicted"],
)

train_preds_df["actual"] = train_labels
train_preds_df["predicted"] = train_preds

valid_preds_df = pd.DataFrame(
    columns=["actual", "predicted"],
)

valid_preds_df["actual"] = valid_labels
valid_preds_df["predicted"] = valid_preds

test_preds_df = pd.DataFrame(
    columns=["actual", "predicted"],
)

test_preds_df["actual"] = test_labels
test_preds_df["predicted"] = test_preds

# save in float format
train_preds_df.to_csv(
    "predictions/efficientnet-train-preds.csv", index=False, float_format="%.3f"
)
valid_preds_df.to_csv(
    "predictions/efficientnet-valid-preds.csv", index=False, float_format="%.3f"
)
test_preds_df.to_csv(
    "predictions/efficientnet-test-preds.csv", index=False, float_format="%.3f"
)

1/1 [==============================] - 0s 429ms/step
